# Crude oil blending
Source: https://github.com/mohiteprathamesh1996/Crude-Blending

Article: https://medium.com/analytics-vidhya/crude-blending-gaining-a-competitive-edge-in-the-oil-gas-industry-through-linear-programming-32c0709c5d6a

#### Load packages

In [1]:
import pandas as pd
import numpy as np
import itertools
#from tqdm import tqdm
from pulp import *
import matplotlib.pyplot as plt
from IPython.display import display, HTML

def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

import warnings
warnings.filterwarnings("ignore")

#### Load data

In [2]:
raw_crude_df = pd.read_excel("Crude-Blending.xlsx", sheet_name="Raw Crude")
raw_crude_df.set_index(["Crude Type"], inplace=True)

products_df = pd.read_excel("Crude-Blending.xlsx", sheet_name="Products")
products_df.set_index(["Product Name"], inplace=True)

display_side_by_side(dfs=[raw_crude_df, products_df], captions=["Raw Crude", "Blended Products from Raw Crude"])

#### Decision variables

In [3]:
crude_types = raw_crude_df.index.to_list()
products = products_df.index.to_list()

var_dict = LpVariable.dicts("Blends", [(crude, product) for crude in crude_types for product in products], lowBound=0, cat="Integer")

#### Model initialization

In [4]:
model = LpProblem("Optimized Blending of Crude Oil Types", LpMaximize)

#### Objective function

In [5]:
purchasing_cost = lpSum([raw_crude_df.loc[c, "Purchasing Price Per Barrel"] * var_dict[(c, p)] for c in crude_types for p in products])
conversion_cost = lpSum([4 * var_dict[(c, p)] for c in crude_types for p in products])
selling_price = lpSum([products_df.loc[p, "Selling Price per Barrel"] * var_dict[(c, p)] for p in products for c in crude_types])
model += selling_price - (purchasing_cost + conversion_cost)

#### Constraints

In [6]:
#1 Up to 7500 barrels of each crude type can be purchased per day
for c in crude_types:
    model += lpSum([var_dict[(c, p)] for p in products]) <= 7500

#2 Fuel quality based on Octane number
for p in products:
    model += lpDot([var_dict[(c,p)] for c in crude_types], [raw_crude_df.loc[c, "Octane Number"] for c in crude_types]) \
             >= products_df.loc[p, "Octane Number"] * lpSum([var_dict[(c, p)] for c in crude_types])
    
#3 Maximum production capacity of 50000 barrels per day
model += lpSum([var_dict[(c, p)] for c in crude_types for p in products]) <= 50000

#4 Fulfill daily customer demand 
for p in products:
    model += lpSum([var_dict[(c,p)] for c in crude_types]) == products_df.loc[p, "Demand"]

#5 Sulfur content limitations
for p in products:
    model += lpDot([var_dict[(c,p)] for c in crude_types], [raw_crude_df.loc[c, "Sulfur content (%)"] for c in crude_types]) \
             <= products_df.loc[p, "Allowable Sulfur (%)"] * lpSum([var_dict[(c, p)] for c in crude_types])

#### Optimal solution

In [7]:
model.solve()

if LpStatus[model.status]=="Optimal":
    optimal_soln = pd.DataFrame([(v.name, int(v.varValue)) for v in model.variables() if v.varValue!=0], columns=["Decisions", "Number of Barrels"])

#### Display optimal blending proportion for each product

In [8]:
print("Total Profit : USD $ {}".format(round(value(model.objective), 2)))

display_side_by_side(
    dfs=[round(100 * np.divide(optimal_soln[optimal_soln["Decisions"].str.contains(p)].set_index("Decisions"),
        optimal_soln[optimal_soln["Decisions"].str.contains(p)].set_index("Decisions").sum()), 2).rename(columns={"Number of Barrels":"Blend_Perc (%)"}) for p in products], 
    captions=["Optimal Blending Proportions for "+i for i in products])

Total Profit : USD $ 186819.97


,Blend_Perc (%)
Decisions,
"Blends_('Brent',_'Fuel_1')",24.530000
"Blends_('Saharan_Blend',_'Fuel_1')",26.370000
"Blends_('West_Texas_Intermediate',_'Fuel_1')",49.100000
,Blend_Perc (%)
Decisions,
"Blends_('Brent',_'Fuel_2')",29.200000
"Blends_('Saharan_Blend',_'Fuel_2')",62.350000
"Blends_('West_Texas_Intermediate',_'Fuel_2')",8.450000
,Blend_Perc (%)
